# Buildings that have ongoing processes

Many buildings have ongoing process that happen over time. This is controlled by an AZ timer function.
```python
@app.function_name(name="factionBuildingTimer")
@app.schedule(schedule="0 */10 * * * *", 
              arg_name="mytimer",
              run_on_startup=RUNNING_LOCALLY)
```



In [116]:
import numpy as np
import pandas as pd
import logging
import sys, os, yaml, ssl, asyncio

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

# mapping to the modules that make the app
sys.path.insert(0, "../..")


In [117]:
ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio

# this is required for running in a Jupyter Notebook.
nest_asyncio.apply()

from app.connectors import cmdb_graph
from app.objects import structures
c = cmdb_graph.CosmosdbClient()

Starting with getting a list of the structures and listing them. 

In [123]:
faction_res = structures.get_faction_pop_structures(c)
pd.DataFrame(faction_res)

,faction,pop,structure,action
0,"{'wealth': 5, 'name': 'An', 'objid': '22650120...","{'isIdle': 'true', 'name': 'An To', 'objid': '...","{'name': 'Oil Well', 'objid': '5461385315509',...",structure
1,"{'wealth': 5, 'name': 'An', 'objid': '22650120...","{'isIdle': 'true', 'name': 'An Mi', 'objid': '...","{'name': 'Solar Panel', 'objid': '281559755258...",structure
2,"{'wealth': 5, 'name': 'An', 'objid': '22650120...","{'isIdle': 'true', 'name': 'An Er', 'objid': '...","{'name': 'Farmland', 'objid': '9272123375134',...",structure
3,"{'wealth': 5, 'name': 'An', 'objid': '22650120...","{'isIdle': 'true', 'name': 'An Ray', 'objid': ...","{'name': 'Farmland', 'objid': '5419403967820',...",structure
4,"{'name': 'Moslettiamac', 'objid': '17676170043...","{'isIdle': 'true', 'name': 'Moslettiamac K', '...","{'name': 'Farmland', 'objid': '7640085175388',...",structure


Here we are attaching `structure` to the message so that the EventHub trigger can catch it and process it. Other than that, it does nothing. 

In [125]:
message = faction_res[0]
message['action']

'structure'

In [126]:
message.keys()

dict_keys(['faction', 'pop', 'structure', 'action'])

In [127]:
message['structure']

{'name': 'Oil Well',
 'objid': '5461385315509',
 'ownedBy': '1852583025275',
 'type': 'oil_well',
 'description': 'Produces energy by extracting hydrocarbons from the ground.',
 'owned_by': 'pop',
 'faction_augments': '{wealth: 1}',
 'requires_attr': '{industry: 0.3}',
 'renews_location_resource': '{hydrocarbons: 10}',
 'effort': '2',
 'render_type': 'mesh',
 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
 'objtype': 'building',
 'id': '5461385315509'}

In [128]:
message['pop']

{'isIdle': 'true',
 'name': 'An To',
 'objid': '1852583025275',
 'conformity': 0.46,
 'literacy': 0.492,
 'aggression': 0.482,
 'constitution': 0.482,
 'health': 0.7,
 'isIn': '2265012067295',
 'industry': 0.482,
 'wealth': 0.487,
 'factionLoyalty': 0.286,
 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
 'objtype': 'pop',
 'id': '1852583025275'}

In [129]:
message['faction']

{'wealth': 5,
 'name': 'An',
 'objid': '2265012067295',
 'lat': -0.117,
 'long': -0.0,
 'infrastructure': 0,
 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
 'objtype': 'faction',
 'id': '2265012067295'}

## faction_augments

In [130]:
structures.augemt_faction(c, message)

EXOADMIN:  wealth has changed from 5 to 6

            g.V().has('label','faction').has('objid','2265012067295').property('wealth', 6)
        


{'wealth': 1}

In [131]:
fac_id = message['faction']['objid']
check_faction = f"""
        g.V().has('label','faction').has('objid','{fac_id}').valueMap()
    """
c.run_query(check_faction)
c.clean_nodes(c.res)

[{'wealth': 6,
  'name': 'An',
  'objid': '2265012067295',
  'lat': -0.117,
  'long': -0.0,
  'infrastructure': 0,
  'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
  'objtype': 'faction',
  'id': '2265012067295'}]

# The ongoing faction process in Azure Functions


In [95]:
messages = structures.get_faction_pop_structures(c)
pd.DataFrame(messages)

,faction,pop,structure,action
0,"{'wealth': 5, 'name': 'An', 'objid': '22650120...","{'isIdle': 'true', 'name': 'An To', 'objid': '...","{'name': 'Oil Well', 'objid': '5461385315509',...",structure
1,"{'wealth': 5, 'name': 'An', 'objid': '22650120...","{'isIdle': 'true', 'name': 'An Mi', 'objid': '...","{'name': 'Solar Panel', 'objid': '281559755258...",structure
2,"{'wealth': 5, 'name': 'An', 'objid': '22650120...","{'isIdle': 'true', 'name': 'An Er', 'objid': '...","{'name': 'Farmland', 'objid': '9272123375134',...",structure
3,"{'wealth': 5, 'name': 'An', 'objid': '22650120...","{'isIdle': 'true', 'name': 'An Ray', 'objid': ...","{'name': 'Farmland', 'objid': '5419403967820',...",structure
4,"{'name': 'Moslettiamac', 'objid': '17676170043...","{'isIdle': 'true', 'name': 'Moslettiamac K', '...","{'name': 'Farmland', 'objid': '7640085175388',...",structure


In [99]:
message = messages[0]
message.keys()

dict_keys(['faction', 'pop', 'structure', 'action'])

In [100]:
message['structure']  

{'name': 'Oil Well',
 'objid': '5461385315509',
 'ownedBy': '1852583025275',
 'type': 'oil_well',
 'description': 'Produces energy by extracting hydrocarbons from the ground.',
 'owned_by': 'pop',
 'faction_augments': '{wealth: 1}',
 'requires_attr': '{industry: 0.3}',
 'renews_location_resource': '{hydrocarbons: 10}',
 'effort': '2',
 'render_type': 'mesh',
 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
 'objtype': 'building',
 'id': '5461385315509'}

In [103]:
print(f"EXOADMIN: process_structure, structure: {message['structure']['name']}: {message['structure']['objid']}")

EXOADMIN: process_structure, structure: Oil Well: 5461385315509


In [104]:
if message['structure']['faction_augments']:
    print(True)

True
